In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

import os
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
train_df = pd.read_csv("data/train.json")
test_df = pd.read_json("data/final_temp.json")

y_test = test_df["interest_level"]

print(test_df.head())

        bathrooms  bedrooms                       building_id  \
10000         1.0         2  c5c8a357cba207596b04d1afd1e4f130   
100013        1.0         4                                 0   
100016        1.0         2  3ba49a93260ca5df92fde024cb4ca61f   
100020        2.0         1  0372927bcb6a0949613ef5bf893bbac7   
100099        2.0         2  39661b8ce46a8d71461497819f67c279   

                    created  \
10000   2016-06-12 12:19:27   
100013  2016-04-28 01:32:41   
100016  2016-04-27 03:19:56   
100020  2016-04-13 06:01:42   
100099  2016-04-06 06:27:51   

                                              description    display_address  \
10000                                                        Columbus Avenue   
100013  Beautifully renovated 3 bedroom flex 4 bedroom...  West 143rd Street   
100016  Stunning unit with a great location and lots o...  West 107th Street   
100020  This huge sunny ,plenty of lights 1 bed/2 bath...   West 21st Street   
100099  Stunning two b

In [3]:
def feature_engineering(df, flag, b_dict={}, m_dict={}):

    ulimit = np.percentile(df.price.values, 99)
    df['price'].loc[df.price > ulimit] = ulimit

    llimit = np.percentile(df.latitude.values, 1)
    ulimit = np.percentile(df.latitude.values, 99)
    df['latitude'].loc[df.latitude < llimit] = llimit
    df['latitude'].loc[df.latitude > ulimit] = ulimit

    llimit = np.percentile(df.longitude.values, 1)
    ulimit = np.percentile(df.longitude.values, 99)
    df['longitude'].loc[df.longitude < llimit] = llimit
    df['longitude'].loc[df.longitude > ulimit] = ulimit
    
    df["created"] = pd.to_datetime(df["created"])
    df["created_year"] = df["created"].dt.year
    df["created_month"] = df["created"].dt.month
    df["created_day"] = df["created"].dt.day
    df["created_hour"] = df["created"].dt.hour
    df["is_night"] = 1*(df["created_hour"] <= 7)
    df["created_weekday"] = df["created"].dt.weekday
    df["is_weekend"] = 1*(df["created_weekday"] > 5)
    
    df["total_rooms"] = df["bedrooms"] + df["bathrooms"]
    df['price_per_bed'] = df['price']/(df['bedrooms']+1)
    df['price_per_bath'] = df['price']/(df['bathrooms']+1)
    df["price_per_room"] = df['price']/(df['total_rooms']+1)
    
    df['description'] = df['description'].str.replace('<[^<>]+>', ' ')
    df['description'] = df['description'].str.replace('[0-9]+', 'num')
    df['description'] = df['description'].str.lower().replace('[^a-zA-Z0-9]', ' ')
    for i in df.index:
        df.loc[i, 'len_description'] = len(df.loc[i, 'description'])
        df.loc[i, 'num_features'] = len(df.loc[i, 'features'])

    

    if not flag:
        interest_dummies = pd.get_dummies(df.interest_level)
        df_dumm = pd.concat([df,interest_dummies[['low','medium','high']]], axis = 1).drop('interest_level', axis = 1)

        df_n = df_dumm[['building_id', 'manager_id', 'low', 'high', 'medium']]
        df_n_m = df_n.groupby(df_n.manager_id).mean()
        df_n_b = df_n.groupby(df_n.building_id).mean()

        df_n_m['manager_skill'] = df_n_m['low']*0+ df_n_m['medium']*1 + df_n_m['high']*2
        df_n_b['build_popularity'] = df_n_b['low']*0+ df_n_b['medium']*1 + df_n_b['high']*2

        df_n_m.drop(['low', 'high', 'medium'], axis=1)
        df_n_b.drop(['low', 'high', 'medium'], axis=1)
        b_dict = df_n_b.to_dict()
        m_dict = df_n_m.to_dict() 

    for i in range(df.shape[0]):
        if df.loc[df.index[i], 'building_id'] in b_dict['build_popularity']:
            df.loc[df.index[i], 'building_popularity'] = b_dict['build_popularity'][df.loc[df.index[i], 'building_id']]
        else:
            df.loc[df.index[i], 'building_popularity'] = 0.33453255354249495 #bp_mean
    
        if df.loc[df.index[i], 'manager_id'] in m_dict['manager_skill']:
            df.loc[df.index[i], 'manager_skill'] = m_dict['manager_skill'][df.loc[df.index[i], 'manager_id']]
        else:
            df.loc[df.index[i], 'manager_skill'] = 0.3666338439324288 #ms_mean
    df["logprice"] = np.log(df["price"])

    num_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "num_features", 
    "len_description","created_year", "created_month", "created_day", "created_hour", "created_weekday", 
    "is_night", "is_weekend", "price_per_bed", "price_per_bath", "total_rooms", "price_per_room",
    "building_popularity", "description", "features"]
    X = df[num_feats]
    return [X, b_dict, m_dict]
train, b_dict, m_dict = feature_engineering(df=train_df,flag=0)
test, b_dict, m_dict = feature_engineering(df=test_df,flag=1, b_dict = b_dict,m_dict =  m_dict)
train["logprice"] = np.log(train_df["price"])
test["logprice"] = np.log(test_df["price"])

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
train["pos"] = train_df.longitude.round(3).astype(str) + '_' + train_df.latitude.round(3).astype(str)
test["pos"] = test_df.longitude.round(3).astype(str) + '_' + test_df.latitude.round(3).astype(str)

vals = train['pos'].value_counts()
dvals = vals.to_dict()
train["density"] = train['pos'].apply(lambda x: dvals.get(x, vals.min()))
test["density"] = test['pos'].apply(lambda x: dvals.get(x, vals.min()))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [5]:
features_to_use=["bathrooms", "bedrooms", "latitude", "longitude", "price", "num_features", 
    "len_description","created_year", "created_month", "created_day", "created_hour", "created_weekday", 
    "is_night", "is_weekend", "price_per_bed", "price_per_bath", "total_rooms", "price_per_room", 
    "building_popularity", "logprice", "density"]

In [6]:
train['description'] = train_df['description'].str.replace('<[^<>]+>', ' ')
train['description'] = train_df['description'].str.replace('[0-9]+', 'num')
train['description'] = train_df['description'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex=True)
test['description'] = test_df['description'].str.replace('<[^<>]+>', ' ')
test['description'] = test_df['description'].str.replace('[0-9]+', 'num')
test['description'] = test_df['description'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex=True)
# from textblob import TextBlob
# train_df['sentiment_polarity'] = train_df['description'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
# train_df['sentiment_subjectivity'] = train_df['description'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)
# test_df['sentiment_polarity'] = test_df['description'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
# test_df['sentiment_subjectivity'] = test_df['description'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)
# features_to_use.append('sentiment_polarity') 
# features_to_use.append('sentiment_subjectivity')


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [7]:
index=list(range(train_df.shape[0]))
random.shuffle(index)
a=[np.nan]*len(train_df)
b=[np.nan]*len(train_df)
c=[np.nan]*len(train_df)

for i in range(5):
    building_level={}
    for j in train_df['manager_id'].values:
        building_level[j]=[0,0,0]
    test_index=index[int((i*train_df.shape[0])/5):int(((i+1)*train_df.shape[0])/5)]
    train_index=list(set(index).difference(test_index))
    for j in train_index:
        temp=train_df.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1
    for j in test_index:
        temp=train_df.iloc[j]
        if sum(building_level[temp['manager_id']])!=0:
            a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
            b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
            c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
train_df['manager_level_low']=a
train_df['manager_level_medium']=b
train_df['manager_level_high']=c

train['manager_level_low'] = train_df['manager_level_low']
train['manager_level_medium'] = train_df['manager_level_medium']
train['manager_level_high'] = train_df['manager_level_high']

a=[]
b=[]
c=[]
building_level={}
for j in train_df['manager_id'].values:
    building_level[j]=[0,0,0]
for j in range(train_df.shape[0]):
    temp=train_df.iloc[j]
    if temp['interest_level']=='low':
        building_level[temp['manager_id']][0]+=1
    if temp['interest_level']=='medium':
        building_level[temp['manager_id']][1]+=1
    if temp['interest_level']=='high':
        building_level[temp['manager_id']][2]+=1

for i in test_df['manager_id'].values:
    if i not in building_level.keys():
        a.append(np.nan)
        b.append(np.nan)
        c.append(np.nan)
    else:
        a.append(building_level[i][0]*1.0/sum(building_level[i]))
        b.append(building_level[i][1]*1.0/sum(building_level[i]))
        c.append(building_level[i][2]*1.0/sum(building_level[i]))
test_df['manager_level_low']=a
test_df['manager_level_medium']=b
test_df['manager_level_high']=c

test['manager_level_low'] = test_df['manager_level_low']
test['manager_level_medium'] = test_df['manager_level_medium']
test['manager_level_high'] = test_df['manager_level_high']

features_to_use.append('manager_level_low') 
features_to_use.append('manager_level_medium') 
features_to_use.append('manager_level_high')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [99]:
print(train_df.manager_level_low)

10        0.740741
100004    0.637500
100007    0.818182
100014    0.906250
100026    0.578947
100027    0.285714
100030    1.000000
10004     1.000000
100044    0.474747
100048    1.000000
10005     1.000000
100051    0.500000
100052    0.444444
100053    1.000000
100055    0.984791
100058    0.984791
100062    0.777778
100063    0.682266
100065    0.992832
100066    0.896018
10007     0.625000
100071    0.904523
100075    0.784946
100076    0.638889
100079    0.109091
100081    0.965517
100083         NaN
100084    0.388889
100085    1.000000
100087    0.956522
            ...   
99903     0.350000
99906     0.000000
9991      0.714286
99915     1.000000
99917     1.000000
99919     0.681115
99921     0.647059
99923     1.000000
99924     0.468085
99931     1.000000
99933     0.458333
99935     1.000000
99937     0.750000
9994      1.000000
99953          NaN
99956     1.000000
99960     0.593750
99961     0.351955
99964     0.353846
99965     1.000000
99982     0.682266
99984     0.

In [19]:
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)
            
train['display_address'] = train_df['display_address']         
train['manager_id'] = train_df['manager_id']
train['building_id'] = train_df['building_id']
train['street_address'] = train_df['street_address']

test['display_address'] = test_df['display_address']         
test['manager_id'] = test_df['manager_id']
test['building_id'] = test_df['building_id']
test['street_address'] = test_df['street_address']

train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))

target_num_map = {'high':0, 'low':1, 'medium':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [42]:
X_count = pd.read_csv("data/X_count.csv")
X_test_count = pd.read_csv("data/X_test_count.csv")

X_count = X_count.drop(['index'], axis=1)
X_test_count = X_test_count.drop(['index'], axis=1)

print(X_count.head())
print(X_test_count.head())

   manager_listings_count  building_listings_count  s_address_listings_count  \
0                      90                        3                         3   
1                     134                       58                        30   
2                     191                       99                        57   
3                     210                       13                         9   
4                      86                       25                        23   

   d_address_listings_count  
0                        22  
1                        69  
2                        73  
3                        59  
4                        23  
   manager_listings_count  building_listings_count  s_address_listings_count  \
0                      86                       24                        21   
1                      15                     8286                         3   
2                       9                       19                        21   
3                  

In [43]:
# features_to_use.append('manager_listings_count')
# features_to_use.append('building_listings_count')
# features_to_use.append('s_address_listings_count')
# features_to_use.append('d_address_listings_count')

train_X = pd.concat([train_X, X_count])
test_X = pd.concat([test_X, X_test_count])

In [31]:
train['features'] = train_df['features']
test['features'] = test_df['features']
# train.head()
# train.to_csv("train_all_feat.csv", index = False)
# test.to_csv("test_all_feat.csv", index = False)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [32]:
print(test.features.head())

10000     D o o r m a n _ E l e v a t o r _ F i t n e s ...
100013                                        P r e - W a r
100016    p r e w a r _ e l e v a t o r _ D o g s _ A l ...
100020    D o o r m a n _ E l e v a t o r _ P r e - W a ...
100099    R o o f _ D e c k _ D i n i n g _ R o o m _ E ...
Name: features, dtype: object


In [24]:
# features_to_use=["bathrooms", "bedrooms", "latitude", "longitude", "price", "num_features", 
#     "len_description","created_year", "created_month", "created_day", "created_hour", "created_weekday", 
#     "is_night", "is_weekend", "price_per_bed", "price_per_bath", "total_rooms", "price_per_room", 
#     "building_popularity", "logprice", "density"]
# count_features = CountVectorizer(stop_words='english', max_features=500,ngram_range=(1,4))
# count_features_train_sparse = count_features.fit_transform(train["features"])
# count_features_test_sparse = count_features.transform(test["features"])

# train_X = sparse.hstack([train[features_to_use], count_features_train_sparse]).tocsr()
# test_X = sparse.hstack([test[features_to_use], count_features_test_sparse]).tocsr()

# tfidf_features = TfidfVectorizer(stop_words='english', max_features=500,ngram_range=(1,4))
# tfidf_features_train_sparse = count_features.fit_transform(train_df["features"])
# tfidf_features_test_sparse = count_features.transform(test_df["features"])

# train_X = sparse.hstack([train_X, tfidf_features_train_sparse]).tocsr()
# test_X = sparse.hstack([test_X, tfidf_features_test_sparse]).tocsr()

train_X = train[features_to_use]
test_X = test[features_to_use]

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [44]:
count_desc = CountVectorizer(stop_words='english', max_features=500,ngram_range=(1,4))
count_desc_train_sparse = count_desc.fit_transform(train_df["description"])
count_desc_test_sparse = count_desc.transform(test_df["description"])

train_X = sparse.hstack([train_X, count_desc_train_sparse]).tocsr()
test_X = sparse.hstack([test_X, count_desc_test_sparse]).tocsr()

tfidf_desc = CountVectorizer(stop_words='english', max_features=500,ngram_range=(1,4))
tfidf_desc_train_sparse = tfidf_desc.fit_transform(train_df["description"])
tfidf_desc_test_sparse = tfidf_desc.transform(test_df["description"])

train_X = sparse.hstack([train_X, tfidf_desc_train_sparse]).tocsr()
test_X = sparse.hstack([test_X, tfidf_desc_test_sparse]).tocsr()
print(features_to_use)

ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 39481, expected 66292.

,bathrooms,bedrooms,latitude,longitude,price,num_features,len_description,created_year,created_month,created_day,...,manager_level_medium,manager_level_high,display_address,manager_id,building_id,street_address,manager_listings_count,building_listings_count,s_address_listings_count,d_address_listings_count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,90.0,3.0,3.0,22.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,134.0,58.0,30.0,69.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,191.0,99.0,57.0,73.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,210.0,13.0,9.0,59.0
4,1.0,1.0,40.7108,-73.95390,2400.0,7.0,464.0,2016.0,6.0,16.0,...,0.371795,0.064103,909.0,2191.0,3934.0,1966.0,86.0,25.0,23.0,23.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,8286.0,1.0,1.0
6,1.0,2.0,40.7513,-73.97220,3800.0,6.0,800.0,2016.0,6.0,1.0,...,0.529412,0.058824,5678.0,2043.0,5486.0,5372.0,21.0,8286.0,16.0,228.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,8286.0,3.0,3.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,138.0,43.0,10.0,17.0
9,1.0,2.0,40.7575,-73.96250,3495.0,6.0,774.0,2016.0,6.0,14.0,...,0.215385,0.046154,5706.0,2747.0,6087.0,9992.0,18.0,8286.0,20.0,56.0


In [22]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=321, num_rounds=1300):
    param = {'booster': 'gbtree',
              #'objective': 'multi:softmax',
              'objective': 'multi:softprob',
              'eval_metric': 'mlogloss',
              'gamma': 2,
              'min_child_weight': 5,
              'max_depth': 5,
              'lambda': 10,
              'alpha':10,
              'subsample': 0.5,
              'colsample_bytree': 0.5,
              'colsample_bylevel': 0.5,
              'eta': 0.03,
              'tree_method': 'exact',
              'seed': 2017,
              "num_class":3,
              'iid':False,
            'silent':0
              }
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=10)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [21]:
def runRF(X_train, y_train, X_test):
    model = RandomForestClassifier(n_estimators=500, max_depth=20, max_features='auto'
    , min_samples_leaf=30, min_samples_split=40, n_jobs=-1, verbose=2)
    print("\nRandom Forest: Training...\n")
    model.fit(X_train, y_train)

    y_train_pred = model.predict_proba(X_train)
    print("Random Forest: Log loss on training set: %f" %log_loss(y_train, y_train_pred))

    print("\nPredicting on test set...\n\n\n")
    y_test_pred = model.predict_proba(X_test)

    return y_test_pred, model

In [25]:
y_test_pred, model = runXGB(train_X, train_y, test_X, num_rounds=1000)
print(y_test_pred[0:5])
y_test = np.array(test_df['interest_level'].apply(lambda x: target_num_map[x]))

print(log_loss(y_test, y_test_pred))


[00:21:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 14 pruned nodes, max_depth=5
[00:21:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 4 pruned nodes, max_depth=5
[00:21:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[00:21:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 14 pruned nodes, max_depth=5
[00:21:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 6 pruned nodes, max_depth=5
[00:21:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=5
[00:21:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 6 pruned nodes, max_depth=5
[00:21:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 2 pruned nodes, max_depth=5
[00:21:38] /wo

[00:21:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 18 pruned nodes, max_depth=5
[00:21:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=5
[00:21:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 12 pruned nodes, max_depth=5
[00:21:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[00:21:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 10 pruned nodes, max_depth=5
[00:21:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 8 pruned nodes, max_depth=5
[00:21:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 2 pruned nodes, max_depth=2
[00:21:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 6 pruned nodes, max_depth=5
[00:21:39] /wor

[00:21:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 14 pruned nodes, max_depth=5
[00:21:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 8 pruned nodes, max_depth=5
[00:21:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 6 pruned nodes, max_depth=5
[00:21:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 4 pruned nodes, max_depth=3
[00:21:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 8 pruned nodes, max_depth=5
[00:21:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 18 pruned nodes, max_depth=5
[00:21:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 10 pruned nodes, max_depth=5
[00:21:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 10 pruned nodes, max_depth=5
[00:21:40] /w

[00:21:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 6 pruned nodes, max_depth=5
[00:21:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 10 pruned nodes, max_depth=5
[00:21:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 4 pruned nodes, max_depth=4
[00:21:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 4 pruned nodes, max_depth=5
[00:21:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 14 pruned nodes, max_depth=4
[00:21:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 18 pruned nodes, max_depth=4
[00:21:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 12 pruned nodes, max_depth=5
[00:21:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 10 pruned nodes, max_depth=5
[00:21:41] 

[00:21:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 18 pruned nodes, max_depth=4
[00:21:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 10 pruned nodes, max_depth=5
[00:21:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 16 pruned nodes, max_depth=5
[00:21:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 12 pruned nodes, max_depth=4
[00:21:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 2 pruned nodes, max_depth=5
[00:21:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 16 pruned nodes, max_depth=5
[00:21:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 8 pruned nodes, max_depth=5
[00:21:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 10 pruned nodes, max_depth=5
[00:21:43]

...

[00:22:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 26 pruned nodes, max_depth=1
[00:22:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 22 pruned nodes, max_depth=3
[00:22:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 22 pruned nodes, max_depth=4
[00:22:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 40 pruned nodes, max_depth=3
[00:22:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 8 pruned nodes, max_depth=4
[00:22:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 30 pruned nodes, max_depth=5
[00:22:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 32 pruned nodes, max_depth=4
[00:22:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 14 pruned nodes, max_depth=2
[00:22:21] /work

[00:22:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 12 pruned nodes, max_depth=4
[00:22:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 36 pruned nodes, max_depth=3
[00:22:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 44 pruned nodes, max_depth=0
[00:22:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 24 pruned nodes, max_depth=5
[00:22:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 32 pruned nodes, max_depth=4
[00:22:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 40 pruned nodes, max_depth=0
[00:22:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 22 pruned nodes, max_depth=1
[00:22:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 42 pruned nodes, max_depth=5
[00:22:22] /w

[00:22:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 26 pruned nodes, max_depth=5
[00:22:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 30 pruned nodes, max_depth=2
[00:22:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 30 pruned nodes, max_depth=0
[00:22:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 22 pruned nodes, max_depth=5
[00:22:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 30 pruned nodes, max_depth=3
[00:22:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 32 pruned nodes, max_depth=0
[00:22:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 26 pruned nodes, max_depth=4
[00:22:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 28 pruned nodes, max_depth=4
[00:22:23] /wo

[00:22:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 28 pruned nodes, max_depth=5
[00:22:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 22 pruned nodes, max_depth=5
[00:22:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 18 pruned nodes, max_depth=4
[00:22:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 24 pruned nodes, max_depth=2
[00:22:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 14 pruned nodes, max_depth=3
[00:22:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 26 pruned nodes, max_depth=4
[00:22:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 40 pruned nodes, max_depth=4
[00:22:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 20 pruned nodes, max_depth=2
[00:22:24] /w

[00:22:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 40 pruned nodes, max_depth=4
[00:22:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 36 pruned nodes, max_depth=4
[00:22:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 24 pruned nodes, max_depth=0
[00:22:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 34 pruned nodes, max_depth=0
[00:22:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 34 pruned nodes, max_depth=1
[00:22:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 24 pruned nodes, max_depth=0
[00:22:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 16 pruned nodes, max_depth=3
[00:22:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 26 pruned nodes, max_depth=4
[00:22:25] /wor

In [22]:
y_test_pred, model = runRF(train_X, train_y, test_X)
y_test = np.array(test_df['interest_level'].apply(lambda x: target_num_map[x]))
print(log_loss(y_test, y_test_pred))


Random Forest: Training...

building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.3s


building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 66 of 500
building tree 67 of 500
building tree 68 of 500
building tree 69 of 500
building tree 70 of 500
building tree 71 of 500
building tree 72 of 500
building tree 73 of 500
building tree 74 of 500
building tree 75 of 500
building tree 76 of 500
building tree 77 of 500
building tree 78

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   23.6s


building tree 159 of 500
building tree 160 of 500
building tree 161 of 500
building tree 162 of 500
building tree 163 of 500
building tree 164 of 500
building tree 165 of 500
building tree 166 of 500
building tree 167 of 500
building tree 168 of 500
building tree 169 of 500
building tree 170 of 500
building tree 171 of 500
building tree 172 of 500
building tree 173 of 500
building tree 174 of 500
building tree 175 of 500
building tree 176 of 500
building tree 177 of 500
building tree 178 of 500
building tree 179 of 500
building tree 180 of 500
building tree 181 of 500
building tree 182 of 500
building tree 183 of 500
building tree 184 of 500
building tree 185 of 500
building tree 186 of 500
building tree 187 of 500
building tree 188 of 500
building tree 189 of 500
building tree 190 of 500
building tree 191 of 500
building tree 192 of 500
building tree 193 of 500
building tree 194 of 500
building tree 195 of 500
building tree 196 of 500
building tree 197 of 500
building tree 198 of 500


...

[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.3min finished
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    3.0s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    4.1s finished


Random Forest: Log loss on training set: 0.563304

Predicting on test set...





[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    1.0s


0.6364178990704258


[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    1.2s finished


In [63]:
imp_feat = np.asarray(model.feature_importances_)
x = np.argsort(imp_feat)[::-1]
print(x[0:20])

[  19   18   17   14   15    4   20   10    6    3    2   21  870   12
  370  775  275   16 1828    1]


In [29]:
xgtrain = xgb.DMatrix(train_X, label=train_y)
y_train_pred = model.predict(xgtrain)
print(log_loss(train_y, y_train_pred))

0.4449174410316288
